In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import tensorflow as tf
try:
    import tensorflow_io as tfio
except:
    !pip install tensorflow-io
    import tensorflow_io as tfio

In [3]:
a1 = tfio.audio.AudioIOTensor('../input/emergency-vehicle-siren-sounds/sounds/ambulance/sound_1.wav')
print(a1)

In [4]:
a10 = tfio.audio.AudioIOTensor('../input/emergency-vehicle-siren-sounds/sounds/ambulance/sound_10.wav')
print(a10)

In [5]:
feature = []
label = []
folders = ['ambulance', 'firetruck', 'traffic']
ambulance = []
firetruck = []
traffic = []
for folder in folders:
    if folder == 'ambulance':
        ambulance = os.listdir('../input/emergency-vehicle-siren-sounds/sounds/'+folder)    
    elif folder =='firetruck':
        firetruck = os.listdir('../input/emergency-vehicle-siren-sounds/sounds/'+folder)      
    else:
        traffic = os.listdir('../input/emergency-vehicle-siren-sounds/sounds/'+folder)

In [7]:
s1=[1,1,0,0,1,1,0,0]
s2=[1,1,1,0,1,1,1,0]
s3=[1,1,1,1,1,1,1,1]
s4=[0.9,0.9,0,0,0.9,0.9,0,0]
s5=[0.8,0.8,0.7,0,0.96,0.79,0.89,0]
s6=[0.69,0.69,0.65,0.8,0.69,0.65,0.69,0.8]

n1=[1,1,0,0,1,1,1,1]
n2=[1,1,1,1,0,0,0,0]
n3=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]

In [8]:
import librosa
import random as rd
ambulanceCount = len(ambulance)
fireTruckCount = len(firetruck)
trafficCount =  len(traffic)
feature = []
label = []
si=[]
ni=[]
#Loads all files in folders and extracts their features
def parser():
    for folder in folders:
        if folder == 'ambulance':
            for i in range(ambulanceCount):
                if ambulance[i] != 'sample.py' and '.png' not in ambulance[i]:
                    X, sample_rate = librosa.load('../input/emergency-vehicle-siren-sounds/sounds/'+folder+'/'+ambulance[i], res_type='kaiser_fast')
                    mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
                    me=np.append(mels,rd.choice([s1,s2,s3,s4,s5,s6]))
                    feature.append(me)
                    label.append(1)
                    
        elif folder =='firetruck':
            for i in range(fireTruckCount):
                if firetruck[i] != 'sample.py'and '.png' not in firetruck[i]:
                    X, sample_rate = librosa.load('../input/emergency-vehicle-siren-sounds/sounds/'+folder+'/'+firetruck[i], res_type='kaiser_fast')
                    mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
                    me=np.append(mels,rd.choice([s1,s2,s3,s4,s5,s6]))
                    feature.append(me)
                    label.append(1)
                    
        else:
            for i in range(trafficCount):
                if (traffic[i] != 'sample.py') and (traffic[i] != "sound_601.wav") and  '.png' not in traffic[i]:
                    X, sample_rate = librosa.load('../input/emergency-vehicle-siren-sounds/sounds/'+folder+'/'+traffic[i], res_type='kaiser_fast')
                    mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
                    me=np.append(mels,rd.choice([n1,n2,n3]))
                    feature.append(me)
                    label.append(0)
                    
            return [feature, label]

In [9]:
t=parser()

In [10]:
t = np.array(t)
d = t.transpose()

In [11]:
from tensorflow.keras.utils import to_categorical 
X_1 = d[:, 0]
Y1 = d[:, 1]
print(X_1.shape, Y1.shape)
X1 = np.empty([600, 136])
Y1 = to_categorical(Y1)

In [13]:
print(X1.shape)
print(Y1.shape)

In [16]:
import random as rd
for i in range(600):
    
    X1[i] = (X_1[i])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, random_state = 1)

In [25]:
m1=tf.keras.Sequential([tf.keras.layers.Dense(136,input_shape=X_train[0].shape),
                              tf.keras.layers.Dense(1024,activation='tanh'),
                              tf.keras.layers.Dense(2048,activation='tanh'),
                              tf.keras.layers.Dense(1024,activation='tanh'),
                              tf.keras.layers.Dense(2,activation='softmax')
                             ])

In [26]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
m1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy', f1_m,precision_m, recall_m])

In [28]:
m1.fit(X_train, Y_train, epochs = 90, batch_size = 50, validation_data = (X_test, Y_test))

In [30]:
predictions = m1.predict(X_test)
score = m1.evaluate(X_test, Y_test)
print(score)

In [31]:
m1.save('model.h5')